In [2]:
import requests
import re 
import os 
import traceback 
from bs4 import BeautifulSoup 
from lxml import etree
from fake_useragent import UserAgent

In [7]:
def getHtmlContent(url):
 UA = UserAgent(verify_ssl = False)   
 headers = {
    "User-Agent": UA.random,
    }

 try:
    r = requests.get(url, headers = headers)
    r.raise_for_status()
    if(r.encoding != "UTF-8"):
        r.encoding = r.apparent_encoding
    return r.text
 except:
    return ""

In [8]:
URL = "https://www.dianping.com/search/keyword/22/0_%E7%BB%98%E6%9C%AC%E9%A6%86"
html = getHtmlContent(URL)
soup = BeautifulSoup(html, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
   <meta content="webkit" name="renderer"/>
   <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
   <style>
    .container {
            width: 998px;
            margin: 0 auto;
        }
        .header {
            height: 70px;
            font-size: 28px;
            color: #999999;
            border-bottom: 3px solid #FFC300;
            overflow: hidden;
            display: none;
        }
        .logo {
            margin-top: 8px;
            margin-left: 1px;
            height: 54px;
            line-height: 54px;
            padding-left: 160px;
            background: url('https://s3plus.meituan.net/v1/mss_f231eb419c414559a1837748d11d4312/yoda-resources/mtlog.png') no-repeat;
            background-size: auto 54px;
        }
        .dpHeader {
            height: 70px;
            font-size: 28px;
 